# Preliminaries

Here, we explain briefly how we organise our notebooks. Throughout all notebooks, we shall employ a custom module:

In [ ]:
import ipynb_utils

It contains a configuration dictionary named `CFG` which holds global environment variables. This shall facilitate the process of storing and loading data between notebooks.

In [ ]:
CFG = ipynb_utils.CFG

padding_length = max([len(k) for k in CFG])

print("CFG Dictionary:")
for k, v in CFG.items():
    print(f"  {k:{padding_length}} : {v}")

Finally, let us delineate the contents of our notebooks and the data upon which they respectively depend:

- `0--preliminaries+retrieval.ipynb`: The present notebook. the code that retrieves the data can be found in the subsequent section.

TODO:
- `1--retrieval.ipynb`: Describes the procedure to download `train.csv` (`CFG["TRAIN_DATA_PATH"]`) and also downloads `airports.csv` (`CFG["AIRPORTS_DATA_PATH"]`). Both files are requisite for the remaining notebook.
- `1-1--eda-basic.ipynb`: Conducts basic exploratory data analysis, producing a processed data frame `df.pkl` (`CFG["PROCESSED_DATA_PATH"]`). This file is necessary for all subsequent notebooks.
- `1-*--eda-add_*.ipynb`: Additional exploratory data analysis, each independent of the others.
- `2-*--model_*.ipynb`: Machine Learning models, each independent of the others.

In the `archive/` directory, the reader may find two additional notebooks:

TODO:
- `0--retrieval_kaggle.ipynb`: Describes the procedure to download the data via kaggle—with appropriate credentials.
- `1--analysis_hint-ignorant.ipynb`: Provides a data analysis under where we pretend to ignore the fact that the instructors meant to insert a duplication.

# Data Retrieval

In this notebook, we shall provide the code required to retrieve the data intended for analysis. It should be noted that appropriate credentials must be supplied in order to access the database.

We shall employ the following modules:

In [ ]:
import os
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv

Global configs:

In [ ]:
DATA_DIR = CFG["DATA_DIR"]

# HTTP address of the (public) repository.
HTTP_REPO_URL = "https://github.com/davidmegginson/ourairports-data"

# Path of the csv file relative to the (remote) repository root.
CSV_PATH_REL = "airports.csv"

# Target path of the csv file on the local machine.
AIRPORTS_DATA_PATH = CFG["AIRPORTS_DATA_PATH"]

# Full HTTP address of the csv file.
http_url = f"{HTTP_REPO_URL}/blob/main/{CSV_PATH_REL}"

# Paths to which dataframes will be saved
DF_PKL_PATH_TAR = os.path.join(DATA_DIR, "df_raw.pkl")
DF_CSV_PATH_TAR = os.path.join(DATA_DIR, "df_raw.csv") 

A brief examination of the given database, conducted within a GUI application such as DBeaver, reveals that the following SQL query yields the correct dataset:

In [ ]:
QUERY = """
    SET SCHEMA 'diabetes';

    SELECT * FROM
            patient p
        LEFT JOIN
            blood_metrics bm
        ON p.id = bm.patientid
        LEFT JOIN
            pedigree_outcome po
        ON p.id = po.patientid
        LEFT JOIN
            skin s
        ON p.id = s.patientid
    ;
"""

We load the credentials required for database access from the `.env` file into Python.

In [ ]:
load_dotenv()

DB_CONFIG = {
    "name": os.getenv("DB_NAME"),
    "scheme": os.getenv("DB_SCHEME"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "host": os.getenv("DB_HOST"),
    "port": os.getenv("DB_PORT")
}

DB_URI = (
    "{scheme}://{user}:{password}@{host}:{port}/{name}"
    .format(**DB_CONFIG)
)

We establish a connection with the database and load the data specified by the query into a pandas data frame.

In [ ]:
db = create_engine(DB_URI)

with db.connect() as conn:
    df = pd.read_sql(QUERY, conn)

Let us confirm that the download process has been successful.

In [ ]:
df.info()

In [ ]:
df.sample(10)


The join operation in the SQL query may cause duplication of columns. Further, the column "patientid" is identical to "id". We shall remove such duplicates immediately.

In [ ]:
df = df.loc[:, ~df.columns.duplicated()]

if "patientid" in df.columns:
    df = df.drop(columns=["patientid"])

At last, the data frame is ready to be stored: To preserve the original data structure, we employ a pickle file; the csv version serves solely for direct visual inspection.

In [ ]:
df.to_pickle(DF_PKL_PATH_TAR)
df.to_csv(DF_CSV_PATH_TAR, index=False)